# Mistral மாதிரிகளுடன் கட்டமைத்தல்

## அறிமுகம்

இந்த பாடத்தில் பின்வருவன கையாளப்படும்:  
- வெவ்வேறு Mistral மாதிரிகளை ஆராய்தல்  
- ஒவ்வொரு மாதிரிக்கும் பயன்பாடுகள் மற்றும் சூழல்களை புரிந்துகொள்வது  
- ஒவ்வொரு மாதிரியின் தனித்துவ அம்சங்களை காட்டும் குறியீட்டு உதாரணங்கள்.


## மிஸ்ட்ரல் மாதிரிகள்

இந்த பாடத்தில், நாங்கள் 3 வெவ்வேறு மிஸ்ட்ரல் மாதிரிகளை ஆராயப்போகிறோம்:  
**மிஸ்ட்ரல் லார்ஜ்**, **மிஸ்ட்ரல் ஸ்மால்** மற்றும் **மிஸ்ட்ரல் நெமோ**.

இந்த மாதிரிகள் ஒவ்வொன்றும் Github மாதிரி சந்தையில் இலவசமாக கிடைக்கின்றன. இந்த நோட்புக் உள்ள குறியீடு இந்த மாதிரிகளை பயன்படுத்தி குறியீட்டை இயக்கும். Github மாதிரிகளை பயன்படுத்தி [AI மாதிரிகளுடன் முன்மாதிரி உருவாக்குதல்](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) குறித்த மேலதிக விவரங்கள் இங்கே உள்ளன.


## Mistral Large 2 (2407)
Mistral Large 2 தற்போது Mistral இன் முக்கிய மாதிரியாகும் மற்றும் நிறுவன பயன்பாட்டிற்காக வடிவமைக்கப்பட்டுள்ளது.

மாதிரி அசல் Mistral Large இன் மேம்படுத்தப்பட்ட பதிப்பாகும், இது 
-  பெரிய Context Window - 128k مقابل 32k 
-  கணித மற்றும் குறியீட்டு பணிகளில் சிறந்த செயல்திறன் - 76.9% சராசரி துல்லியம் مقابل 60.4% 
-  அதிகரிக்கப்பட்ட பன்மொழி செயல்திறன் - மொழிகள்: ஆங்கிலம், பிரஞ்சு, ஜெர்மன், ஸ்பானிஷ், இத்தாலியன், போர்ச்சுகீஸ், டச்சு, ரஷியன், சீன, ஜப்பானீஸ், கொரியன், அரபிக் மற்றும் இந்தி.

இந்த அம்சங்களுடன், Mistral Large சிறப்பாக செயல்படுகிறது 
- *Retrieval Augmented Generation (RAG)* - பெரிய context window காரணமாக
- *Function Calling* - இந்த மாதிரியில் உள்ளூர் செயல்பாட்டு அழைப்புகள் உள்ளன, இது வெளிப்புற கருவிகள் மற்றும் API களுடன் ஒருங்கிணைப்பை அனுமதிக்கிறது. இந்த அழைப்புகள் ஒரே நேரத்தில் அல்லது தொடர் முறையில் ஒன்றுக்குப் பிறகு ஒன்று செய்யப்படலாம்.
- *Code Generation* - இந்த மாதிரி Python, Java, TypeScript மற்றும் C++ உருவாக்கத்தில் சிறப்பாக செயல்படுகிறது.


### மிஸ்ட்ரல் லார்ஜ் 2 பயன்படுத்தி RAG உதாரணம்


இந்த உதாரணத்தில், ஒரு உரை ஆவணத்தில் RAG மாதிரியை இயக்க Mistral Large 2 பயன்படுத்தப்படுகிறது. கேள்வி கொரிய மொழியில் எழுதப்பட்டு, ஆசிரியர் கல்லூரிக்கு முன் செய்த செயல்கள் பற்றி கேட்கிறது.

இது Cohere Embeddings மாதிரியை பயன்படுத்தி உரை ஆவணத்திற்கும் கேள்விக்கும் embeddings உருவாக்குகிறது. இந்த மாதிரிக்காக, faiss Python தொகுப்பை வெக்டர் சேமிப்பாக பயன்படுத்துகிறது.

Mistral மாதிரிக்கு அனுப்பப்படும் முன்மொழிவு கேள்விகளையும் கேள்விக்கு ஒத்துள்ள மீட்டெடுக்கப்பட்ட துண்டுகளையும் உள்ளடக்கியது. பின்னர் மாதிரி இயற்கை மொழி பதிலை வழங்குகிறது.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small என்பது Mistral குடும்பத்தின் மற்றொரு மாதிரி ஆகும், இது premier/enterprise வகையில் உள்ளது. பெயர் கூறுவது போல், இந்த மாதிரி ஒரு சிறிய மொழி மாதிரி (SLM) ஆகும். Mistral Small பயன்படுத்துவதன் நன்மைகள்: 
- Mistral LLMs போன்ற Mistral Large மற்றும் NeMo உடன் ஒப்பிடுகையில் செலவு குறைவு - 80% விலை குறைப்பு
- குறைந்த தாமதம் - Mistral இன் LLMs உடன் ஒப்பிடுகையில் விரைவான பதில்
- நெகிழ்வானது - தேவையான வளங்களில் குறைந்த கட்டுப்பாடுகளுடன் வெவ்வேறு சூழல்களில் பயன்படுத்தக்கூடியது. 


Mistral Small சிறந்தது: 
- சுருக்கம், உணர்ச்சி பகுப்பாய்வு மற்றும் மொழிபெயர்ப்பு போன்ற உரை அடிப்படையிலான பணிகளுக்கு. 
- செலவு குறைவான காரணத்தால் அடிக்கடி கோரிக்கைகள் செய்யப்படும் பயன்பாடுகளுக்கு 
- குறைந்த தாமதம் கொண்ட குறியீட்டு பணிகள், உதாரணமாக மதிப்பாய்வு மற்றும் குறியீட்டு பரிந்துரைகள்


## Mistral சிறிய மற்றும் Mistral பெரியவை ஒப்பிடுதல்

Mistral சிறிய மற்றும் பெரியவை இடையேயான தாமத வேறுபாடுகளை காண கீழே உள்ள செல்களை இயக்கவும்.

3-5 வினாடிகள் இடையேயான பதில் நேர வேறுபாடு காணப்பட வேண்டும். அதே கேள்விக்கு பதிலின் நீளம் மற்றும் பாணியையும் கவனிக்கவும்.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

இந்த பாடத்தில் விவாதிக்கப்பட்ட மற்ற இரண்டு மாதிரிகளுடன் ஒப்பிடுகையில், Mistral NeMo என்பது Apache2 உரிமத்துடன் கூடிய ஒரே இலவச மாதிரி ஆகும்.

இது முந்தைய திறந்த மூல LLM ஆகிய Mistral 7B இன் மேம்படுத்தப்பட்ட பதிப்பாக பார்க்கப்படுகிறது.

NeMo மாதிரியின் சில பிற அம்சங்கள்:

- *மேலும் திறமையான டோக்கனைசேஷன்:* இந்த மாதிரி பொதுவாக பயன்படுத்தப்படும் tiktoken ஐவிட Tekken டோக்கனரை பயன்படுத்துகிறது. இது பல மொழிகள் மற்றும் குறியீடுகளுக்கு மேம்பட்ட செயல்திறனை வழங்குகிறது.

- *மிகைப்படுத்தல்:* அடிப்படை மாதிரி மிகைப்படுத்தலுக்காக கிடைக்கிறது. இது மிகைப்படுத்தல் தேவைப்படும் பயன்பாடுகளுக்கு அதிகமான நெகிழ்வுத்தன்மையை வழங்குகிறது.

- *நேட்டிவ் செயல்பாடு அழைப்பு* - Mistral Large போல, இந்த மாதிரியும் செயல்பாடு அழைப்பில் பயிற்சி பெற்றுள்ளது. இது திறந்த மூல மாதிரிகளில் முதன்மையான ஒன்றாக தனித்துவமாக்குகிறது.


## Mistral NeMo

இந்த பாடத்தில் விவாதிக்கப்பட்ட மற்ற இரண்டு மாதிரிகளுடன் ஒப்பிடுகையில், Mistral NeMo என்பது Apache2 உரிமத்துடன் கூடிய ஒரே இலவச மாதிரி ஆகும்.

இது Mistral இன் முந்தைய திறந்த மூல LLM ஆகும் Mistral 7B க்கு மேம்படுத்தல் என பார்க்கப்படுகிறது.

NeMo மாதிரியின் சில பிற அம்சங்கள்:

- *மேலும் திறமையான டோக்கனைசேஷன்:* இந்த மாதிரி பொதுவாக பயன்படுத்தப்படும் tiktoken க்கு பதிலாக Tekken டோக்கனைசரை பயன்படுத்துகிறது. இது பல மொழிகள் மற்றும் குறியீடுகளுக்கு மேம்பட்ட செயல்திறனை வழங்குகிறது.

- *மிகைப்படுத்தல்:* அடிப்படை மாதிரி மிகைப்படுத்தலுக்கு கிடைக்கிறது. இது மிகைப்படுத்தல் தேவைப்படும் பயன்பாட்டு வழிகளுக்கு அதிகமான நெகிழ்வுத்தன்மையை வழங்குகிறது.

- *நேட்டிவ் செயல்பாடு அழைப்பு* - Mistral Large போல, இந்த மாதிரியும் செயல்பாடு அழைப்பில் பயிற்சி பெற்றுள்ளது. இது திறந்த மூல மாதிரிகளில் முதன்மையான ஒன்றாக இதை தனித்துவமாக்குகிறது.


### டோக்கனர்களை ஒப்பிடுதல்

இந்த மாதிரியில், Mistral NeMo எப்படி டோக்கனீசேஷனை கையாள்கிறது என்பதை Mistral Large உடன் ஒப்பிடுவோம்.

இரு மாதிரிகளும் ஒரே பிராம்ப்டை எடுத்துக் கொள்கின்றன, ஆனால் NeMo Mistral Large உடன் ஒப்பிடுகையில் குறைவான டோக்கன்களை திருப்பி அளிக்கும் என்பதை நீங்கள் காணலாம்.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## கற்றல் இங்கே நிறுத்தப்படாது, பயணத்தை தொடருங்கள்

இந்த பாடத்தை முடித்த பிறகு, உங்கள் Generative AI அறிவை மேம்படுத்த [Generative AI கற்றல் தொகுப்பை](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) பாருங்கள்!


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**குறிப்பு**:  
இந்த ஆவணம் AI மொழிபெயர்ப்பு சேவை [Co-op Translator](https://github.com/Azure/co-op-translator) மூலம் மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சித்தாலும், தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனிக்கவும். அசல் ஆவணம் அதன் சொந்த மொழியில் அதிகாரப்பூர்வ மூலமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பின் பயன்பாட்டால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பேற்கமாட்டோம்.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
